GRUPO: 12

Francisco de Borja Lozano del Moral

Manuel Ortega Salvador

# Práctica 2

Vamos a ver en la práctica los conceptos vistos en clase sobre Recomendadores.
- En la primera parte veremos algunos conceptos de la librería Pandas de python ejemplificados en lo que hemos llamado Recomendador Simple, que ordena y muestra las películas en el TOP de popularidad pero no realiza recomendaciones personalizadas, es decir, a todos los usuarios les recomienda la misma lista de películas (como en el TOP 10 de Netflix).
- En la segunda parte veremos un recomendador personalizado basado en contenidos y 
- En la tercera parte veremos los recomendadores por filtrado colaborativo usando la librería SURPRISE y haciendo hincapié en como evaluar los resultados de la recomendación. 


## Primera parte. Recomendacion simple por popularidad

Esta primera parte está inspirada en Rounak Banik, 2018 Recommender Systems in Python: Beginner Tutorial DE DATACAMP.COM
Vamos a utilizar los dos datasets de MovieLens que se pueden descargar de: 
https://www.kaggle.com/rounakbanik/the-movies-dataset

- The Full Dataset: Consists of 26,000,000 ratings and 750,000 tag applications applied to 45,000 movies by 270,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

- The Small Dataset: Comprises of 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users.

En esta primera parte solo tienes que ejecutar las celdas del tutorial y responder las preguntas del final. No se pide implementar nada extra

### Recomendador simple basado en popularidad

Vamos a realizar una version simple de un Recomendador que ofrece recomendaciones generalizadas a todos los usuarios según la popularidad de la película y el género. La idea básica detrás de este recomendador es que las películas que son más populares y más aclamadas por la crítica tendrán una mayor probabilidad de gustar a la audiencia promedio. Este modelo no da recomendaciones personalizadas en base a los gustos de un usuario concreto.

La implementación de este modelo muy sencilla. Basta ordenar las películas según las calificaciones y la popularidad y mostrar las mejores películas de nuestra lista. Como paso adicional, podemos pasar un argumento de género para filtrar y mostrar solo las mejores películas de un género en particular. 

Los pasos que vamos a realizar en esta primera versión son los siguientes:
 - Decidir cual es el criterio que nos permite ordenar las peliculas recomendadas. 
 - Calcular esa métrica para cada pelicula de la lista total de películas.
 - Ordenar las películas y mostrar las mejores.

Cargamos los datos del archivo csv de peliculas usando pandas.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Pandas es un paquete de Python que proporciona estructuras de datos tabulares con columnas de tipo heterogéneo con etiquetas en columnas y filas. Estas tablas se conocen como DataFrames. Un dataframe es una estructura de datos con dos dimensiones en la cual se puede guardar datos de distintos tipos (como caractéres, enteros, valores de punto flotante, factores y más) en columnas. Es similar a una hoja de cálculo o una tabla de SQL

In [5]:
# Importamos Pandas
import pandas as pd
import warnings; 
warnings.simplefilter('ignore')  # para evitar algunos warnings al cargar los datos.

# Load Movies Metadata in a dataframe
md = pd.read_csv('./data/movies_metadata.csv', low_memory=False)

# se puede imprimir solo las primeras 3 filas para ver como son los datos.
md.head(3)
#md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Utilizo las clasificaciones de The Movie Database (TMDb)  para crear nuestra tabla de películas principales. 
Vamos a utilizar la fórmula de calificación ponderada de IMDB Internet Movie Database.

Calificación ponderada o Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
dónde,
- v es el número de votos para la película
- m son los votos mínimos requeridos para aparecer en la tabla
- R es la calificación promedio de la película
- C es el voto medio

El siguiente paso es determinar un valor apropiado para m, los votos mínimos requeridos para aparecer en la tabla. Usaremos el percentil 95 como nuestro límite. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 95% de las películas de la lista.

Vamos a hacer una tabla Top 250 por popularidad y otra para seleccionar las películas de un genero en particular.  


In [6]:
from ast import literal_eval
import numpy as np
import matplotlib.pyplot as plt

In [7]:
## obtenemos la media de las puntuaciones de las películas de la tabla. 
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [8]:
# cuantas cumplen el percentil 0,95
m = vote_counts.quantile(0.95)
m

434.0

Para que una película sea elegida tiene que tener por los menos 434 votos en TMDB (95%). Tambien vemos que la media de rating de las peliculas en TMDB es de 5.244 en una escala de 10. Hay 2274 peliculas que cumplen estos criterios.

In [9]:
# Utilizo la columna release_date para construir una columna year, porque solo me interesa el año. 
# No es necesario comprender todos los detalles pero puedes consultar la documentación de panda.

md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [10]:
# limpiamos la lista de generos para que se quede solo con el nombre en vez de la estructura [{'id': 16, 'name': 'Animation'}. Sólo ejecutar una vez. 
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

Vamos a mostrar el resultado despues de modificar la columna genres y añadir la columna year para el año. 

In [11]:
md.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


In [12]:

seleccionadas = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
seleccionadas.shape
# shape devuelve el numero filas y columnas de la tabla de peliculas seleccionadas. 

(2274, 6)

In [13]:
seleccionadas.head(3)

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]"


In [14]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [15]:
seleccionadas['wr'] = seleccionadas.apply(weighted_rating, axis=1)

In [16]:
seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)

###  TOP 15 de peliculas

In [17]:
seleccionadas.head(15)
#mostramos el TOP 15

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


Vemos que tres películas de Christopher Nolan:  **Inception**, **The Dark Knight** e **Interstellar** son las preferidas por popularidad en la parte superior de nuestra lista de TOP 15. En la tabla se ve un fuerte sesgo de los usuarios de TMDB hacia géneros y directores particulares, pero de los sesgos no hablaremos en esta práctica. 

Vamos a construir una funcion que permite filtrar y construir una tabla de peliculas recomendadas para generos particulares. 

In [18]:
# Separamos las peliculas que tienen varios generos de forma que aparezcan varias filas de la misma película.
# una por cada genero.
# De nuevo no hace falta entender todos los detalles. Sólo observar el comportamiento en la tabla resultado.

s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

gen_md = md.drop('genres', axis=1).join(s)

In [19]:
# Metemos todo el código anterior en una función.
# Vamos a relajar nuestras condiciones predeterminadas al percentil 85 en lugar de 95 para tener más generos.
    
def construye_tabla(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    seleccionadas = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
    seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
    
    seleccionadas['wr'] = seleccionadas.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)
    
    return seleccionadas

Veamos nuestro método en acción mostrando las 15 mejores películas románticas. Con esto forzamos el genero que nos interesa y resolvemos el problema de que el romance casi no figuraba en nuestro Top  genérico a pesar de ser uno de los géneros de películas más populares.

### TOP Películas románticas

In [20]:
construye_tabla('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


Hasta aquí lo único que hemos hecho es ordenar el catalogo. Las recomendaciones son generales (no personalizadas) por popularidad, pero lo que nosotros queremos hacer son recomendaciones personalizadas para un usuario del que sabemos su perfil, sus preferencias, alguna película que le ha gustado.
La siguiente parte corresponde al recomendador basado en contenido que dada una película (puede ser usada como perfil) recomienda peliculas similares. En la tercera parte veremos el filtrado colaborativo.

Para el Recomendador simple hemos usado las películas del dataset completo, mientras que para el sistema de recomendación personalizado haremos uso del segundo data set (Small)

### Ejercicio parte 1.

Realiza algunas pruebas de recomendaciones por popularidad y comenta los resultados obtenidos.  Incluye los resultados y los comentarios en este archivo.

¿Te parece útil esta forma de recomendación de contenidos? 
¿Sería útil también para otras aplicaciones, por ejemplo, para recomendar libros o restaurantes?




#### Mismas películas lideran varios leaderboards

Como este sistema se basa principalmente en popularidad, la misma película va a aparecer con muy alta probabilidad en torno a la misma posición en todos los géneros que se le atribuyen. Por ejemplo, Inception aparece como mejor película de Acción, SciFi, Thriller y Mystery. No se tiene en cuenta la relevancia real dentro del género ni lo querida que es dentro de los posibles amantes de estos géneros. Esto hace que, a la hora de recomendar películas de un género, las recomendaciones pueden llegar a ser consideradas incorrectas, ya que no se tiene en cuenta los géneros principales, si no todos los que tienen etiquetados.


In [21]:
md[md['original_title'] == 'Inception']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
15480,False,NaN,160000000,"[Action, Thriller, Science Fiction, Mystery, A...",http://inceptionmovie.warnerbros.com/,27205,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",...,825532764.0,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075.0,2010


In [22]:
construye_tabla('Action').head(1)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.955099


In [23]:
construye_tabla('Science Fiction').head(1)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.939069


In [24]:
construye_tabla('Thriller').head(1)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.95646


In [25]:
construye_tabla('Mystery').head(1)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.966674


#### Es lo reciente es más popular?

Cada año, más gente se puede permitir ver películas, luego la popularidad del cine crece anualmente. Esto parece corresponderse con los porcentajes de las películas totales. Al compararlo con las recomendaciones, se puede ver que el orden cambia mucho. Representación anual de películas en las recomendaciones. Como se puede ver, el uso de los votos ponderados no disminuye mucho la correlación entre el año de salida y la probabilidad de que sea recomendada, y se sigue viendo que las películas recientes están sobrerepresentadas en el ranking, aunque los años estén un poco barajados.

In [26]:
# Distribución de las recomendaciones por años
seleccionadas['year'].value_counts().head(25)/len(seleccionadas)*100

2014    10.4
2013     6.4
2016     5.6
2015     5.2
2009     4.4
2010     4.4
2012     4.0
2001     4.0
2006     3.2
2011     3.2
2004     2.8
2008     2.8
2002     2.8
1999     2.4
1995     2.0
1997     2.0
1998     2.0
2003     2.0
1994     2.0
2007     2.0
2000     1.6
2017     1.6
1988     1.6
2005     1.6
1984     1.2
Name: year, dtype: float64

In [27]:
# Distribución real por años
md['year'].value_counts().head(20)/len(md)*100

2014    4.341706
2015    4.189944
2013    4.154753
2012    3.787446
2011    3.666476
2016    3.527911
2009    3.488321
2010    3.301368
2008    3.239784
2007    2.903268
2006    2.793296
2005    2.474376
2004    2.181850
2002    1.990498
2003    1.939911
2001    1.902521
2000    1.735363
1999    1.590199
1998    1.588000
1997    1.453834
Name: year, dtype: float64

In [28]:

seleccionadas_popularity = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
seleccionadas_popularity['vote_count'] = seleccionadas_popularity['vote_count'].astype('int')

In [29]:
seleccionadas_popularity.sort_values('vote_count', ascending=False)['year'].value_counts().head(20)/len(seleccionadas_popularity)*100

2013    6.464380
2016    6.332454
2014    6.112577
2015    6.068602
2011    5.189094
2010    5.101143
2012    4.705365
2009    4.573439
2008    4.045734
2007    3.913808
2004    3.737907
2006    3.430079
2005    3.386104
2003    2.946350
2002    2.770449
2001    2.506596
2000    2.418646
1998    2.066843
1999    2.066843
1997    1.934916
Name: year, dtype: float64

#### Posible Feedback Loop

Si tiene demasiado peso la popularidad, entonces la gente solo vería películas ya famosas, lo cual las hace aún más famosas. 
Esto puede llegar a ser un problema, ya que desaparecerían de las recomendaciones todas las películas que no hicieron ningún impacto all aparecer en cartelera.
 
Como se puede observar, aun usando votos ponderados, hay 140/250 películas en común entre el resultado ponderado y simplemente conger lo más famoso.

In [30]:
construye_tabla('Action').head(10)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.955099
12481,The Dark Knight,2008,12269,8,123.167259,7.948610
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.929579
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.924031
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.918382
256,Star Wars,1977,6778,8,42.149697,7.908327
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.896841
4135,Scarface,1983,3017,8,11.299673,7.802046
9430,Oldboy,2003,2000,8,10.616859,7.711649
1910,Seven Samurai,1954,892,8,15.01777,7.426145


In [31]:
def construye_tabla_nvotos(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    seleccionadas = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
    seleccionadas = seleccionadas.sort_values('vote_count', ascending=False).head(250)
    
    return seleccionadas

In [72]:
construye_tabla_nvotos('Action').head(10)

,title,year,vote_count,vote_average,popularity
15480,Inception,2010,14075,8.1,29.108149
12481,The Dark Knight,2008,12269,8.3,123.167259
14551,Avatar,2009,12114,7.2,185.070892
17818,The Avengers,2012,12000,7.4,89.887648
26564,Deadpool,2016,11444,7.4,187.860492
23753,Guardians of the Galaxy,2014,10014,7.9,53.291601
26553,Mad Max: Fury Road,2015,9629,7.3,29.36178
18252,The Dark Knight Rises,2012,9263,7.6,20.58258
2458,The Matrix,1999,9079,7.9,33.366332
20830,Iron Man 3,2013,8951,6.8,23.721243


In [73]:
s1 = construye_tabla('Action')['title']
s2 = construye_tabla_nvotos('Action')['title']

print(len(s1))
print(len(s2))
intersection = pd.DataFrame(list(set(s1) & set(s2)))
intersection

250
250


,0
0,I Am Legend
1,Scarface
2,V for Vendetta
3,Lucy
4,X-Men: Days of Future Past
...,...
135,The Bourne Supremacy
136,The Lord of the Rings: The Two Towers
137,Star Wars: Episode I - The Phantom Menace
138,The Lord of the Rings: The Fellowship of the Ring


####  Poco efecto de los percentiles usados
Cabe destacar que, salvo para percentiles muy altos, no hay mucho cambio al modificar el percentil de votos, sobre todo en las posiciones más altas de la lista, como puede apreciarse en los ejemplos siguientes. Esto vuelve a mostrar la importancia de la popularidad a la hora de aparecer en el ranking.

In [32]:
# Metemos todo el código anterior en una función.
# Vamos a relajar nuestras condiciones predeterminadas al percentil 85 en lugar de 95 para tener más generos.
    
def construye_tabla(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    seleccionadas = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
    seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
    
    seleccionadas['wr'] = seleccionadas.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)
    
    return seleccionadas

In [33]:
construye_tabla('Animation', 0.95).head(50)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.583403
5481,Spirited Away,2001,3968,8,41.048867,7.461435
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.155849
2884,Princess Mononoke,1997,2041,8,17.166725,7.153848
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.067935
13724,Up,2009,7048,7,19.330884,6.818411
30315,Inside Out,2015,6737,7,23.985587,6.811413
15472,Despicable Me,2010,6595,7,22.274502,6.808035
12704,WALL·E,2008,6439,7,16.088366,6.804182
6232,Finding Nemo,2003,6292,7,25.497794,6.800407


In [34]:
construye_tabla('Animation', 0.85).head(50)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.909339
5481,Spirited Away,2001,3968,8,41.048867,7.875933
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.772103
2884,Princess Mononoke,1997,2041,8,17.166725,7.771305
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.735274
40251,Your Name.,2016,1030,8,34.461252,7.589820
5553,Grave of the Fireflies,1988,974,8,0.010902,7.570962
19901,Paperman,2012,734,8,7.198633,7.465676
39386,Piper,2016,487,8,11.243161,7.285132
20779,Wolf Children,2012,483,8,10.249498,7.281198


In [35]:
construye_tabla('Animation', 0.25).head(50)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.997288
5481,Spirited Away,2001,3968,8,41.048867,7.996228
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.992708
2884,Princess Mononoke,1997,2041,8,17.166725,7.992680
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.991369
40251,Your Name.,2016,1030,8,34.461252,7.985543
5553,Grave of the Fireflies,1988,974,8,0.010902,7.984718
19901,Paperman,2012,734,8,7.198633,7.979768
39386,Piper,2016,487,8,11.243161,7.969652
20779,Wolf Children,2012,483,8,10.249498,7.969405


#### Conclusión

El efecto del uso de los votos ponderados hacen que las recomendaciones no descarten películas menos vistas, aunque quizás requiere algunos ajustes para disminuir más el efecto de la popularidad, ya que aunque la calidad y popularidad están correlacionadas, las películas recientes de multinacionales tienen una sobre-representación en los rankings.

Aun así, este tipo de recomendación, si se arreglara el primer problema apuntado (géneros posiblemente erroneos), los usuarios serían recomendados con gran posibilidad películas de alta calidad del género escogido.


Un problema causado por usar recomendaciones no personalizadas de este tipo es que no tienen en cuenta cuáles han sido vistas. El público objetivo de este tipo de ranking es gente que no sabe qué película ver, porque no encuentra ninguna. Pero este tipo de persona, con alta probabilidad, ha visto todas las películas famosas del género. Y como se ha apuntado, las películas famosas abundan en el ranking. Por lo tanto, la mayoría de las recomendaciones hechas por este recomendador serían inútiles para la gente que busca recomendaciones.


## Segunda parte.  Recomendacion basada en contenido


El recomendador que construimos en la sección anterior da la misma recomendación a todos, independientemente del gusto personal del usuario. Si una persona que le gustan las películas románticas (y odia la acción) mirara nuestro Top 15, probablemente no le gustarían la mayoría de las películas. Si esta persona mira nuestras listas por género, no estaría recibiendo las mejores recomendaciones.

Por ejemplo, considere a una persona que le gustan: "Dilwale Dulhania Le Jayenge", "My Name is Khan" y "Kabhi Khushi Kabhi Gham". Si miramos los datos de estas películas podríamos inferir que a esta persona le gusta el actor Shahrukh Khan y el director Karan Johar. Incluso si él / ella tuviera acceso a la tabla de películas románticas no las encontraría como las principales recomendaciones, porque no son películas populares. 

Para personalizar más nuestras recomendaciones, voy a crear un motor que calcule la similitud entre películas en función de ciertas métricas y sugiera películas que son más similares a una película en particular que le gustó a un usuario. Como hemos visto en teoría este tipo de recomendaciones se conocen como **recomendación o filtrado basado en contenido**.

Vamos a ver cómo construir dos recomendadores basados en contenido:

- Usando los resúmenes textuales de las películas y taglines. Los taglines son las frases cortas que  suelen aparecer en el cartel de la película, acompañando al título y a una imagen destacada y representativa de la historia que presenta.
- Usando el reparto de películas, equipo, palabras clave y género



In [40]:
# usaremos el conjunto pequeño de películas por eficiencia. 
# Leemos los datos y limpiamos algunos valores nulos.
links_small = pd.read_csv('./data/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
## selecciono las peliculas de links_small con los id de la tabla de metadatos 
smd = md[md['id'].isin(links_small)]
smd.shape
## shape devuelve el numero de filas y columnas. 
## (9099, 25)
# Tenemos *9099* películas disponibles en nuestro conjunto de datos de metadatos de películas pequeñas, que es 5 veces más pequeño que nuestro conjunto de datos original de 45000 películas.

(9099, 25)

In [41]:
smd.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


Tenemos que medir la similitud de las películas con las preferencias del usuario. En este caso usaremos como preferencia una pelicula que le haya gustado antes para recuperar (por similitud) otras películas parecidas.  Elegimos usar similitud usando el atributo de descripción (textual) y el lema (tambien textual). Para calcular la similitud entre dos textos usamos el modelo del espacio vectorial.

In [42]:
smd['tagline'] = smd['tagline'].fillna('')
#El método fillna permite sustituir los valores nulos de una estructura pandas

smd['description'] = smd['overview'] + smd['tagline']
# creamos un campo llamado descrition con la 
smd['description'] = smd['description'].fillna('')

In [43]:
smd.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...


In [44]:
print(smd['description'])

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
40224    From the mind behind Evangelion comes a hit la...
40503    The band stormed Europe in 1963, and, in 1964,...
44821    When Molly Hale's sadness of her father's disa...
44826    All your favorite Pokémon characters are back,...
45265    While holidaying in the French Alps, a Swedish...
Name: description, Length: 9099, dtype: object


#### Similitud del coseno 

Utilizamos el vectorizador TF-IDF
Para calcular un valor de similitud se puede usar ** cosine_similarity** o ** linear_kernel ** de sklearn. La segunda es más eficiente. 
https://scikit-learn.org/stable/modules/metrics.html

La medida de similitud del coseno calcula un valor numerico que representa la similitud entre dos vectores de términos. En este caso los vectores representan los textos de la descripción normalizados.

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [46]:
tfidf_matrix.shape
# filas, columnas

(9099, 268124)

In [47]:
#cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [48]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

Ahora tenemos una matriz de similitud de coseno por pares para todas las películas en nuestro conjunto de datos. El siguiente paso es escribir una función que devuelva las 30 películas más similares según la puntuación de similitud del coseno.

In [49]:
smd = smd.reset_index() 
#reset_index resets the index column) to a regular column and uses a numeric sequence as new index.

titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
#Un objeto "Series" es un vector con datos indexados

In [50]:
print(len(indices))
indices.head(3)

9099


title
Toy Story           0
Jumanji             1
Grumpier Old Men    2
dtype: int64

In [51]:
def get_recommendations(title, N=30):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:1+N]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [52]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

###  Ejercicio parte 2: 
Prueba algunas consultas y comenta los resultados. Compara los resultados con el tipo de recomendaciones realizadas en la parte 1. 
    
    

###  Resultados a primera vista
Este caso es más interesante que el del primer apartado, ya que aquí sí que se busca obtener una recomendación personalizada en función de los gustos del usuario, en lugar de una lista genérica de películas que comparten una característica común como el género. Así obtenemos una lista más variada de películas similares a otra que nos haya gustado. Sin embarbo, vemos que utilizar únicamente la sinopsis de las películas como criterio para calcular la similitud entre ellas no da necesariamente buenos resultados.

Se puede apreciar que en caso de películas con secuelas o pertenecientes a una saga, algunas de ellas sí que aparecen en la lista de recomendados en las posiciones más altas. Sin embargo, vemos en todas las pruebas que la mayoría de las películas de la lista no son nada similares a la introducida inicialmente. Véase el ejemplo de Mi vecino Totoro, que se nos recomiendan películas de terror y drama, que poco tienen que ver con ella. Por ejemplo, en ese caso habría tenido más sentido que se recomendaran películas del mismo director u otras también de animación japonesa, y del mismo modo en Toy Story tendría más sentido que las películas más similares fueran de animación. Por ello vemos que sería necesario tener en cuenta más características de las películas a la hora de calcular la distancia entre ellas, y no sólamente la sinopsis, ya que en la práctica no nos da generalmente buenos resultados.
    
    

In [53]:
get_recommendations('My Neighbor Totoro').head(10)

3504    Final Fantasy: The Spirits Within
7744                      Red Riding Hood
5749                         The Landlord
8821                          Poltergeist
7755                            Insidious
4951                 What's New Pussycat?
8199                      Our Little Girl
3787                       Female Trouble
1596             The Cat from Outer Space
1219                         Fathers' Day
Name: title, dtype: object

####  Sobre-simplificación del problema
Los vectores pertenecientes a las películas parecen no tener en cuenta connotaciones o temas de la película. Parece que solo usan palabras de manera literal. Esto lo podemos ver viendo las recomendaciones para gente que le ha gustado Inception y The Dark Knight y Avatar. 

Salvo para Batman, que practicamente solo recomienda más Batman, para Inception y Avatar, las recomendaciones no tienen apenas nada en común, y a mi parecer, son malas recomendaciones. Se han usado demasiados pocos datos de las películas para crear una representación numérica, se necesitaría que el vector encodificara también el género, actores, directores, etc. 

In [98]:
get_recommendations('Inception').head(10)

5239                              Cypher
141                                Crumb
6398                         Renaissance
653                            Lone Star
1703                               House
4739                    The Pink Panther
319                                 Cobb
2828    What Ever Happened to Baby Jane?
8867                     Pitch Perfect 2
979          Once Upon a Time in America
Name: title, dtype: object

In [99]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [103]:
get_recommendations('Avatar').head(10)

2059                                    The Matrix
4506                              Tears of the Sun
4695    Lara Croft Tomb Raider: The Cradle of Life
2910               Pandora and the Flying Dutchman
538                          Hellraiser: Bloodline
7460                                    Green Zone
7587                                  The American
3015                                 House Party 2
2561                                     Supernova
975                                A Grand Day Out
Name: title, dtype: object

In [114]:
list(smd[smd['original_title'] == 'Lone Star']['description'])[0]

'When the skeleton of his murdered predecessor is found, Sheriff Sam Deeds unearths many other long-buried secrets in his Texas border town.John Sayles invites you to return to the scene of the crime.'

In [113]:
list(smd[smd['original_title'] == 'Inception']['description'])[0]

'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.Your mind is the scene of the crime.'

####  Lenta renovación de las recomendaciones
El problema es la poca variabilidad de la lista de recomendados, ya que depende totalmente de la puntuación, por lo que una película quedará estancada en una posición y solo podrá descender si es superada por otras peliculas estrenadas en años posteriores. 

Esto es un problema, ya que con el tiempo sería más y más improbable que la lista cambie, ya que estará llena por películas muy populares de muy alta calidad que no pueden ser superadas.

####  Buen buscador por criterio
De todos modos puede resultar interesante si el usuario quiere ver una categoría concreta (género, idioma, año de estreno...), por lo que también puede ser aplicable en otras aplicaciones como música (género, idioma, artista...) y restaurantes (tipo de comida, ubicación...).

Se podría solucionar el problema anterior aplicando filtros por año, ya que al disminuir la competición, será más sencillo que películas aparezcan en las recomendaciones.

####  Mejor recomendador para exploración
Como se explicó en la parte anterior, suelen buscar recomendaciones gente que ya ha agotado las fuentes comunes. Aunque poco este recomendador es demaisado poco refinado, es mucho más adecuado a la hora de recomendar nuevo contenido no visto que el anterior.

Además, tiene potencial como filtro de búsqueda, si se quiere algo parecido a un grupo de películas, este metodo se podría usar fácilmente de esta manera:

In [137]:
def get_recommendations(title, N=30):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:1+N]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

N = 500
s1 = set(get_recommendations('Supernova', N))
s2 = set(get_recommendations('Avatar', N))
s3 = set(get_recommendations('Fight Club', N))
intersection = pd.DataFrame(list(s1 & s2 & s3))
intersection

,0
0,The Matrix
1,Virus
2,Armageddon
3,The Hidden


####  Conclusión & Comparación
Este recomendador parece más útil y tiene más potencial, pero se requiere un mejor encodificador. Probablemente, si usaramos un encodificador muy bueno, funcionaría mejor si las descripciones de las películas más largas. Pero, si se usa este tipo de encodificador, probablemente sea mejor crear descripciones de cero que solo usen keywords claves para la trama y el estilo de la película, como una lista de tags, ya que pararía de haber recomendaciones que no tienen nada que ver pero usan lenguaje parecido en la descripción.